In [1]:
# In the notebook:
%pip install -q ipywidgets==8.* jupyterlab_widgets


Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch, json

path = "../outputs/bert/best_model"
tok = AutoTokenizer.from_pretrained(path)
model = AutoModelForSequenceClassification.from_pretrained(path)
id2label = json.load(open(f"{path}/id2label.json"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [3]:
import torch
import numpy as np
import torch.nn.functional as F

MAX_LEN = 128
model.eval()  # inference mode

def predict_one(text, k=3):
    enc = tok([text], truncation=True, padding=True, max_length=MAX_LEN, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**enc).logits
        probs = F.softmax(logits, dim=-1).cpu().numpy()[0]

    # id2label keys may be strings in your JSON; handle both
    def lbl(i): 
        return id2label[str(i)] if isinstance(next(iter(id2label.keys())), str) else id2label[i]

    pred_id = int(np.argmax(probs))
    topk_ids = np.argsort(-probs)[:k]
    topk = [(lbl(i), float(probs[i])) for i in topk_ids]
    return lbl(pred_id), float(probs[pred_id]), topk

# Example usage:
label, p, topk = predict_one("AUTOPILOT DISCONNECTED UNEXPECTEDLY DURING CRUISE; INSPECTION FOUND LOOSE CONNECTOR")
print("Predicted:", label, f"({p:.3f})")
print("Top-k:", topk)


Predicted: AVIONICS/ELECTRICAL (0.965)
Top-k: [('AVIONICS/ELECTRICAL', 0.9653291702270508), ('FLIGHT_CONTROLS', 0.016656190156936646), ('CABIN', 0.005767402704805136)]


In [6]:
# Simple notebook UI: type text → click Predict
from ipywidgets import Textarea, Button, Output, VBox, HBox, IntSlider
from IPython.display import display
import torch.nn.functional as F
import numpy as np
import torch

model.eval()

out = Output()
ta = Textarea(
    value="AUTOPILOT DISCONNECTED UNEXPECTEDLY DURING CRUISE; INSPECTION FOUND LOOSE CONNECTOR",
    placeholder="Type a discrepancy here...",
    layout=dict(width="100%", height="120px")
)
topk = IntSlider(value=3, min=1, max=len(id2label), step=1, description="Top-k")
btn = Button(description="Predict", button_style="primary")

def on_click(_):
    out.clear_output()
    text = ta.value.strip()
    if not text:
        with out: print("Please enter some text.")
        return
    try:
        label, prob, tk = predict_one(text, k=topk.value)
        with out:
            print(f"Predicted: {label} ({prob:.3f})")
            print("Top-k:")
            for lbl, p in tk:
                print(f"  {lbl:>25}  {p:.3f}")
    except Exception as e:
        with out: print("Error:", e)

btn.on_click(on_click)
ui = VBox([ta, HBox([topk, btn]), out])
display(ui)
